In [12]:
import sys
import warnings
sys.path.append('../')
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 1
%aimport utils, GroupingAlgorithm, HEEM_VQE_Functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.auto import tqdm

from qiskit import IBMQ, Aer, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.ignis.verification import get_ghz_simple
from qiskit.compiler import transpile
from qiskit.tools.monitor import job_monitor

from utils import get_backend_connectivity, molecules, Label2Chain, from_string_to_numbers
from GroupingAlgorithm import groupingWithOrder, TPBgrouping
from HEEM_VQE_Functions import measure_circuit_factor, probability2expected, probability2expected_parallel, post_process_results

In [14]:
def compute_energy(qi, circuits, diagonals, factors, method, n_chunks=25):

    counts = []

    counter = 0
    n_runs = int(np.ceil(len(circuits) / n_chunks))
    
    with open('progress_' + method + '.txt', 'w') as f:
            f.write('{}/{}\n'.format(0, len(circuits)))
    
    pbar = tqdm(total=len(circuits), desc='Simulating circuits for {}'.format(method))
    while len(counts) < len(circuits):
        initial = counter * n_chunks
        final = min((counter + 1) * n_chunks, len(circuits))
         
        # Some time IBMQ is returning some random error [5204], so if this happens
        while True:
            try:
                qi.set_config(job_tags=[f'{counter + 1}/{n_runs}'])
                counts_temp = qi.execute(circuits[initial:final]).get_counts()
                break
            except (KeyboardInterrupt, SystemExit):
                return None
            except Exception as e:
                print(e)
                print('Trying again ...')
                pass

        counts += counts_temp
        counter += 1
        
        with open('progress_' + method + '.txt', 'w') as f:
            f.write('{}/{}\n'.format(final, len(circuits)))            
        pbar.update(final - initial)
    pbar.close()
    
    if len(circuits) == 1:
        counts = [counts]
        
    probabilities = [post_process_results(counts[j], circuits[j].num_clbits, qi.run_config.shots) for j in
                     range(len(counts))]
    energy = 0
    
    for j in range(len(probabilities)):
        energy += np.sum((diagonals[j] * np.array(factors[j])[:, None]) @ probabilities[j])
        # energy += np.sum(prob2Exp[j] @ probabilities[j])

    return energy

In [15]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    IBMQ.load_account()

provider_main = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider_CSIC = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')

name_backend = 'ibmq_montreal'
backend = provider_CSIC.get_backend(name_backend)
WC_device = get_backend_connectivity(backend)

G_device = nx.Graph()
G_device.add_edges_from(WC_device)

ibmqfactory.load_account:WARNING:2022-05-12 15:34:07,816: Credentials are already in use. The existing account in the session will be replaced.


In [16]:
NUM_SHOTS = 2 ** 14

backend_hpc = provider_main.get_backend('ibmq_qasm_simulator')
backed_simulation = provider_main.get_backend('simulator_statevector')
simulator = Aer.get_backend('aer_simulator')  # Backend for simulation

device = QasmSimulator.from_backend(backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

backed_calculations = simulator

In [17]:
molecule_name = 'LiH'

try:
    qubit_op = np.load('../data/big_molecules.npy', allow_pickle=True).item()[molecule_name]
    print('Data loaded')
except KeyError:
    print('Computing molecule')
    qubit_op = molecules(molecule_name)
        
paulis, coeffs, labels = Label2Chain(qubit_op)

n_qubits = qubit_op.num_qubits
state_0 = QuantumCircuit(n_qubits)
state_0 = state_0.compose(get_ghz_simple(n_qubits, measure=False))  # Initialized in the GHZ state

print(f'{len(paulis)} total Pauli strings')

Computing molecule
100 total Pauli strings


# Exact (only valid for GHZ)

In [18]:
diagonals = {'I': np.array([1, 0, 0, 1]), 'X': np.array([0, 1, 1, 0]),
             'Y': np.array([0, 1j, -1j, 0]), 'Z': np.array([1, 0, 0, -1])}

energy_exact = 0
for label, coeff in zip(labels, coeffs):
    diagonal = np.ones(4, dtype=complex)
    for op in label:
        diagonal *= diagonals[op]
    energy_exact += coeff * np.sum(diagonal) / 2
    
print('Exact energy: {}'.format(energy_exact))

Exact energy: (-0.42764080030360263+0j)


## Exact: Simulation

# TPB

In [30]:
_, Groups_TPB, Measurements_TPB = TPBgrouping(paulis, print_progress=True)

max_size = 250
print('Number of groups', len(Groups_TPB))
n_runs = int(np.ceil(len(Groups_TPB) / max_size))

prob2Exp_TPB = []
for i in range(n_runs):
    initial = i * max_size
    final = min((i + 1) * max_size, len(Groups_TPB))
    
    prob2Exp_TPB += probability2expected_parallel(-1, coeffs, labels, Groups_TPB[initial:final], Measurements_TPB[initial:final],
                                                  print_progress=True, shift=False, binary=False)

diagonals_TPB = [temp[0] for temp in prob2Exp_TPB]
factors_TPB = [temp[1] for temp in prob2Exp_TPB]

circuits_TPB = [measure_circuit_factor(measurement, n_qubits, measure_all=False).compose(state_0, front=True) for measurement
                 in Measurements_TPB]

Computing Pauli graph:   0%|          | 0/100 [00:00<?, ?it/s]

Number of groups 25


Computing diagonal factors:   0%|          | 0/25 [00:00<?, ?it/s]

In [31]:
method='TPB'
qi_TPB = QuantumInstance(backend=backed_calculations, coupling_map=coupling_map, noise_model=noise_model,
                         basis_gates=basis_gates, shots=NUM_SHOTS // len(Groups_TPB))

qi_TPB._RUN_CONFIG += ["job_name", "job_tags"]
qi_TPB.set_config(job_name=f'{molecule_name} / {method}')

energy_TPB = compute_energy(qi_TPB, circuits_TPB, diagonals_TPB, factors_TPB, method=method)
relative_TPB = np.abs((energy_TPB - energy_exact) / energy_exact)

print('TPB energy: {}'.format(energy_TPB))
print('Relative error: {:.3f} %'.format(relative_TPB * 100))
print(f'Simulation time {qi_TPB.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_TPB))

Simulating circuits for TPB:   0%|          | 0/25 [00:00<?, ?it/s]

TPB energy: -0.43611295449779247
Relative error: 1.981 %
Simulation time 0.5522315502166748 s


# EM

In [26]:
Groups_EM, Measurements_EM, layout_EM = groupingWithOrder(paulis, connected=True, print_progress=True)

max_size = 50
print('Number of groups', len(Groups_EM))
n_runs = int(np.ceil(len(Groups_EM) / max_size))

prob2Exp_EM = []
for i in range(n_runs):
    initial = i * max_size
    final = min((i + 1) * max_size, len(Groups_EM))
    
    prob2Exp_EM += probability2expected_parallel(-1, coeffs, labels, Groups_EM[initial:final], Measurements_EM[initial:final],
                                                  print_progress=True, shift=False, binary=False)

diagonals_EM = [temp[0] for temp in prob2Exp_EM]
factors_EM = [temp[1] for temp in prob2Exp_EM]
    
circuits_EM = [measure_circuit_factor(measurement, n_qubits, measure_all=False).compose(state_0, front=True) for measurement
                 in Measurements_EM]

Computing Pauli graph:   0%|          | 0/100 [00:00<?, ?it/s]

Computing grouping:   0%|          | 0/100 [00:00<?, ?it/s]

Number of groups 11


Computing diagonal factors:   0%|          | 0/11 [00:00<?, ?it/s]

In [27]:
method = 'EM'
qi_EM = QuantumInstance(backend=backed_calculations, coupling_map=coupling_map, noise_model=noise_model,
                        basis_gates=basis_gates, shots=NUM_SHOTS // len(Groups_EM))
qi_EM.set_config(initial_layout=layout_EM[::-1])

qi_EM._RUN_CONFIG += ["job_name", "job_tags"]
qi_EM.set_config(job_name=f'{molecule_name} / {method}')

energy_EM = compute_energy(qi_EM, circuits_EM, diagonals_EM, factors_EM, method=method)
relative_EM= np.abs((energy_EM - energy_exact) / energy_exact)

print('EM energy: {}'.format(energy_EM))
print('Relative error: {:.3f} %'.format(relative_EM * 100))
print(f'Simulation time {qi_EM.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_EM))

Simulating circuits for EM:   0%|          | 0/11 [00:00<?, ?it/s]

EM energy: -0.4398563580673711
Relative error: 2.856 %
Simulation time 0.3066246509552002 s


# HEEM

In [28]:
Groups_HEEM, Measurements_HEEM, layout_HEEM = groupingWithOrder(paulis, G_device, connected=True, print_progress=True)

max_size = 250
print('Number of groups', len(Groups_HEEM))
n_runs = int(np.ceil(len(Groups_HEEM) / max_size))

prob2Exp_HEEM = []
for i in range(n_runs):
    initial = i * max_size
    final = min((i + 1) * max_size, len(Groups_HEEM))
    
    prob2Exp_HEEM += probability2expected_parallel(-1, coeffs, labels, Groups_HEEM[initial:final], Measurements_HEEM[initial:final],
                                                  print_progress=True, shift=False, binary=False)

diagonals_HEEM = [temp[0] for temp in prob2Exp_HEEM]
factors_HEEM = [temp[1] for temp in prob2Exp_HEEM]
    
circuits_HEEM = [measure_circuit_factor(measurement, n_qubits, measure_all=False).compose(state_0, front=True) for measurement
                 in Measurements_HEEM]

Computing Pauli graph:   0%|          | 0/100 [00:00<?, ?it/s]

Computing grouping:   0%|          | 0/100 [00:00<?, ?it/s]

Number of groups 11


Computing diagonal factors:   0%|          | 0/11 [00:00<?, ?it/s]

In [29]:
method = 'HEEM'
qi_HEEM = QuantumInstance(backend=backed_calculations, coupling_map=coupling_map, noise_model=noise_model,
                          basis_gates=basis_gates, shots=NUM_SHOTS // len(Groups_HEEM))
qi_HEEM.set_config(initial_layout=layout_HEEM[::-1])

qi_HEEM._RUN_CONFIG += ["job_name", "job_tags"]
qi_HEEM.set_config(job_name=f'{molecule_name} / {method}')

energy_HEEM = compute_energy(qi_HEEM, circuits_HEEM, diagonals_HEEM, factors_HEEM, method=method, n_chunks=100)
relative_HEEM = np.abs((energy_HEEM - energy_exact) / energy_exact)

print('HEEM energy: {}'.format(energy_HEEM))
print('Relative error: {:.3f} %'.format(relative_HEEM * 100))
print(f'Simulation time {qi_HEEM.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_HEEM))

Simulating circuits for HEEM:   0%|          | 0/11 [00:00<?, ?it/s]

HEEM energy: -0.43616664261264043
Relative error: 1.994 %
Simulation time 0.27233409881591797 s
